# Análise exploratória e teste amostral - linha LECD50

**TODO:**
- (1) Tabela pré-reprocessamento
- (2) Gráficos da análise exploratória
- (3) Tabela pós-reprocessamento

Preparar o ambiente:

In [1]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 25)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


# (1) Tabela pré-reprocessamento

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [2]:
servico = "LECD50" # 605 a partir de 2023-06-01 em diante

#### 1 - Importar e tratar os dados da amostra

In [3]:
amostra = pd.read_excel('../data/raw/Relatorio de Viagens das linhas 605_LECD50. 663 e 844.xlsx')

# padronizar as colunas da amostra 
amostra = amostra.rename(
    columns={"Data": "data",
             "Hora_Inicio": "hora_inicio",
             "Linha": "servico",
             "Veiculo": "id_veiculo",
             "Hora_Fim": "hora_fim",
             "Sentido": "sentido"
             }
    )

# setar tipos de dados e criar colunas com o datetime
amostra['servico'] = amostra['servico'].astype(str)
amostra['data'] = amostra['data'].astype(str)
amostra['hora_inicio'] = amostra['hora_inicio'].astype(str)
amostra['datetime_partida'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_inicio'])
amostra['hora_fim'] = amostra['hora_fim'].astype(str)
amostra['datetime_chegada'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_fim'])
amostra['id_veiculo'] = amostra['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
amostra_tratada =  amostra[amostra['servico'] == "605 (LECD50)"]

amostra_tratada['servico'] = "LECD50"

amostra_tratada

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada
24,2022-09-13,LECD50,27684,06:05:00,06:26:00,I,2022-09-13 06:05:00,2022-09-13 06:26:00
25,2022-09-13,LECD50,27684,06:27:00,06:51:00,V,2022-09-13 06:27:00,2022-09-13 06:51:00
26,2022-09-13,LECD50,27684,09:55:00,10:22:00,I,2022-09-13 09:55:00,2022-09-13 10:22:00
27,2022-09-13,LECD50,27684,10:24:00,10:49:00,V,2022-09-13 10:24:00,2022-09-13 10:49:00
28,2022-09-13,LECD50,27684,10:54:00,11:21:00,I,2022-09-13 10:54:00,2022-09-13 11:21:00
...,...,...,...,...,...,...,...,...
123,2023-07-20,LECD50,27632,08:19:00,08:44:00,V,2023-07-20 08:19:00,2023-07-20 08:44:00
124,2023-07-20,LECD50,27632,08:51:00,09:16:00,I,2023-07-20 08:51:00,2023-07-20 09:16:00
125,2023-07-20,LECD50,27632,09:17:00,09:46:00,V,2023-07-20 09:17:00,2023-07-20 09:46:00
126,2023-07-20,LECD50,27632,09:52:00,10:15:00,I,2023-07-20 09:52:00,2023-07-20 10:15:00


In [4]:
amostra_tratada_2022 = amostra_tratada[amostra_tratada['data'] <= '2022-12-31']
amostra_tratada_2023 = amostra_tratada[amostra_tratada['data'] > '2022-12-31']
datas = amostra_tratada['data'].unique()
veiculos = amostra_tratada['id_veiculo'].unique()

print("A quantidade total de viagens na amostra é:", amostra_tratada.shape[0])
print("A quantidade de viagens na amostra em 2022 é:", amostra_tratada_2022.shape[0])
print("A quantidade de viagens na amostra em 2023 é:", amostra_tratada_2023.shape[0])
print("A amostra é referente aos dias:", datas)
print("A amostra tem dados dos seguintes veículos:", veiculos)

A quantidade total de viagens na amostra é: 104
A quantidade de viagens na amostra em 2022 é: 78
A quantidade de viagens na amostra em 2023 é: 26
A amostra é referente aos dias: ['2022-09-13' '2022-09-21' '2022-09-29' '2023-07-04' '2023-07-12'
 '2023-07-20']
A amostra tem dados dos seguintes veículos: ['27684' '27632']


#### 2 - Remover viagens sobrepostas da amostra

Esta etapa classifica como "Viagem inválida - sobreposição de viagem" aquelas viagens em que um mesmo `id_veiculo` realiza múltiplas viagens em um mesmo intervalo de tempo (`datetime_partida` e `datetime_chegada`).

No caso de duas ou mais viagens sobrepostas, apenas uma será classificada nas demais etapas de classificação do notebook.

In [5]:
amostra_deduplicada = remove_overlapping_trips(amostra_tratada)
amostra_deduplicada

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
24,2022-09-13,LECD50,27684,06:05:00,06:26:00,I,2022-09-13 06:05:00,2022-09-13 06:26:00,NaN
25,2022-09-13,LECD50,27684,06:27:00,06:51:00,V,2022-09-13 06:27:00,2022-09-13 06:51:00,NaN
26,2022-09-13,LECD50,27684,09:55:00,10:22:00,I,2022-09-13 09:55:00,2022-09-13 10:22:00,NaN
27,2022-09-13,LECD50,27684,10:24:00,10:49:00,V,2022-09-13 10:24:00,2022-09-13 10:49:00,NaN
28,2022-09-13,LECD50,27684,10:54:00,11:21:00,I,2022-09-13 10:54:00,2022-09-13 11:21:00,NaN
...,...,...,...,...,...,...,...,...,...
123,2023-07-20,LECD50,27632,08:19:00,08:44:00,V,2023-07-20 08:19:00,2023-07-20 08:44:00,NaN
124,2023-07-20,LECD50,27632,08:51:00,09:16:00,I,2023-07-20 08:51:00,2023-07-20 09:16:00,NaN
125,2023-07-20,LECD50,27632,09:17:00,09:46:00,V,2023-07-20 09:17:00,2023-07-20 09:46:00,NaN
126,2023-07-20,LECD50,27632,09:52:00,10:15:00,I,2023-07-20 09:52:00,2023-07-20 10:15:00,NaN


In [6]:
count_non_nan = amostra_deduplicada['status'].notna().sum()
count_non_nan

0

### 3 Comparar dados da amostra com os dados das viagens completas

#### 3.1 Consultar se as viagens da amostra foram feitas pelos veículos indicados na amostra

In [7]:
# separar as datas da amostra para fazer a query
datas = amostra_deduplicada['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado IN ('605', 'LECD50')
       """
       
viagem_completa_prod = bd.read_sql(q, from_file=True)
viagem_completa_prod.info() 

viagem_completa_prod['id_veiculo'] = viagem_completa_prod['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagem_completa_prod['servico_informado'] = viagem_completa_prod['servico_informado'].astype(str)
viagem_completa_prod['datetime_partida'] = pd.to_datetime(viagem_completa_prod['datetime_partida'])
viagem_completa_prod['datetime_chegada'] = pd.to_datetime(viagem_completa_prod['datetime_chegada'])

viagem_completa_prod      

Downloading: 100%|██████████| 45/45 [00:00<00:00, 130.06rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         45 non-null     object        
 1   servico_informado  45 non-null     object        
 2   sentido            45 non-null     object        
 3   datetime_partida   45 non-null     datetime64[ns]
 4   datetime_chegada   45 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 1.9+ KB


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,27632,605,C,2023-07-04 07:13:57,2023-07-04 08:01:57
1,27632,605,C,2023-07-04 08:19:57,2023-07-04 09:09:57
2,27632,605,C,2023-07-04 09:24:57,2023-07-04 10:07:57
3,27632,605,C,2023-07-04 06:14:27,2023-07-04 06:50:27
4,27640,LECD50,C,2022-09-29 16:47:08,2022-09-29 17:36:38
...,...,...,...,...,...
40,27684,LECD50,C,2022-09-21 16:19:32,2022-09-21 17:12:02
41,27684,LECD50,C,2022-09-21 17:31:32,2022-09-21 18:17:02
42,27684,LECD50,C,2022-09-21 21:05:32,2022-09-21 21:39:32
43,27684,LECD50,C,2022-09-21 12:12:02,2022-09-21 13:02:32


In [8]:
datas

array(['2022-09-13', '2022-09-21', '2022-09-29', '2023-07-04',
       '2023-07-12', '2023-07-20'], dtype=object)

Esta etapa cruza dados da amostra com as viagens completas para o serviço LECD50 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 10 minutos do datetime_partida que consta no amostra.

In [9]:
viagem_completa_prod['datetime_partida'] = pd.to_datetime(viagem_completa_prod['datetime_partida'])
amostra_classificada_v_completa = check_trips(amostra_deduplicada, viagem_completa_prod, 10, 
                                              "Viagem circular identificada e já paga")
amostra_classificada_v_completa

Não existem casos duplicados no cruzamento de dados.


,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2022-09-13,LECD50,27684,06:05:00,06:26:00,I,2022-09-13 06:05:00,2022-09-13 06:26:00,NaN,NaN,NaN,NaN,NaT,NaT
1,2022-09-13,LECD50,27684,06:27:00,06:51:00,V,2022-09-13 06:27:00,2022-09-13 06:51:00,NaN,NaN,NaN,NaN,NaT,NaT
2,2022-09-13,LECD50,27684,09:55:00,10:22:00,I,2022-09-13 09:55:00,2022-09-13 10:22:00,NaN,NaN,NaN,NaN,NaT,NaT
3,2022-09-13,LECD50,27684,10:24:00,10:49:00,V,2022-09-13 10:24:00,2022-09-13 10:49:00,NaN,NaN,NaN,NaN,NaT,NaT
4,2022-09-13,LECD50,27684,11:23:00,11:48:00,V,2022-09-13 11:23:00,2022-09-13 11:48:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2023-07-20,LECD50,27632,06:11:00,06:29:00,I,2023-07-20 06:11:00,2023-07-20 06:29:00,Viagem circular identificada e já paga,27632,605,C,2023-07-20 06:13:58,2023-07-20 06:46:58
100,2023-07-20,LECD50,27632,07:01:00,07:21:00,I,2023-07-20 07:01:00,2023-07-20 07:21:00,Viagem circular identificada e já paga,27632,605,C,2023-07-20 07:03:58,2023-07-20 07:39:58
101,2023-07-20,LECD50,27632,07:55:00,08:18:00,I,2023-07-20 07:55:00,2023-07-20 08:18:00,Viagem circular identificada e já paga,27632,605,C,2023-07-20 07:57:28,2023-07-20 08:39:58
102,2023-07-20,LECD50,27632,08:51:00,09:16:00,I,2023-07-20 08:51:00,2023-07-20 09:16:00,Viagem circular identificada e já paga,27632,605,C,2023-07-20 08:54:28,2023-07-20 09:41:28


### 4 Comparar dados da amostra com os dados das viagens conformidade

In [10]:
q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado IN ('605', 'LECD50')
       """
       
viagem_conformidade = bd.read_sql(q, from_file=True)

viagem_conformidade['id_veiculo'] = viagem_conformidade['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagem_conformidade['servico_informado'] = viagem_conformidade['servico_informado'].astype(str)
viagem_conformidade['datetime_partida'] = pd.to_datetime(viagem_conformidade['datetime_partida'])
viagem_conformidade['datetime_chegada'] = pd.to_datetime(viagem_conformidade['datetime_chegada'])

Downloading: 100%|██████████| 47/47 [00:00<00:00, 142.85rows/s]


In [11]:
amostra_classificada_v_conformidade = check_trips(amostra_classificada_v_completa, viagem_conformidade, 
                                                  10, "Viagem circular inválida - não atendeu ao percentual de conformidade do GPS ou do trajeto")
amostra_classificada_v_conformidade

Não existem casos duplicados no cruzamento de dados.


,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2022-09-13,LECD50,27684,10:54:00,11:21:00,I,2022-09-13 10:54:00,2022-09-13 11:21:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 11:02:51,2022-09-13 11:42:51
1,2022-09-13,LECD50,27684,11:53:00,12:19:00,I,2022-09-13 11:53:00,2022-09-13 12:19:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 11:59:51,2022-09-13 12:38:51
2,2022-09-13,LECD50,27684,12:45:00,13:13:00,I,2022-09-13 12:45:00,2022-09-13 13:13:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 12:52:21,2022-09-13 13:34:51
3,2022-09-13,LECD50,27684,13:49:00,14:23:00,I,2022-09-13 13:49:00,2022-09-13 14:23:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 13:56:30,2022-09-13 14:47:00
4,2022-09-13,LECD50,27684,14:59:00,15:26:00,I,2022-09-13 14:59:00,2022-09-13 15:26:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 15:06:00,2022-09-13 15:51:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2023-07-20,LECD50,27632,07:22:00,07:44:00,V,2023-07-20 07:22:00,2023-07-20 07:44:00,NaN,NaN,NaN,NaN,NaT,NaT
100,2023-07-20,LECD50,27632,08:19:00,08:44:00,V,2023-07-20 08:19:00,2023-07-20 08:44:00,NaN,NaN,NaN,NaN,NaT,NaT
101,2023-07-20,LECD50,27632,09:17:00,09:46:00,V,2023-07-20 09:17:00,2023-07-20 09:46:00,NaN,NaN,NaN,NaN,NaT,NaT
102,2023-07-20,LECD50,27632,10:17:00,10:40:00,V,2023-07-20 10:17:00,2023-07-20 10:40:00,NaN,NaN,NaN,NaN,NaT,NaT


In [12]:
def check_circular_trips(df):
    # Copiando o DataFrame para não alterar o original
    result = df.copy()

    # Ordenar o DataFrame
    result.sort_values(by=['datetime_partida_amostra', 'id_veiculo_amostra'], inplace=True)

    # Criar colunas deslocadas (shifted)
    result['prev_sentido'] = result.groupby('id_veiculo_amostra')['sentido_amostra'].shift(1)
    result['prev_datetime_chegada'] = result.groupby('id_veiculo_amostra')['datetime_chegada_apurado'].shift(1)
    result['prev_status'] = result.groupby('id_veiculo_amostra')['status'].shift(1)

    # Criar a condição e atualizar a coluna 'status'
    condition = (
        (result['prev_sentido'] == 'I') & 
        ((result['datetime_partida_amostra'] - result['prev_datetime_chegada']).dt.total_seconds() < 240) &
        pd.isna(result['status'])
    )

    result.loc[condition, 'status'] = result.loc[condition, 'prev_status']

    # Agora, para as colunas terminadas em "apurado", copie-as também para a linha de baixo (n) quando a condição acima for satisfeita
    cols_to_copy = [col for col in result.columns if col.endswith('apurado')]
    for col in cols_to_copy:
        result[f'prev_{col}'] = result.groupby('id_veiculo_amostra')[col].shift(1)
        result.loc[condition, col] = result.loc[condition, f'prev_{col}']

    # Remover as colunas temporárias
    temp_cols = ['prev_sentido', 'prev_datetime_chegada', 'prev_status'] + [f'prev_{col}' for col in cols_to_copy]
    result.drop(temp_cols, axis=1, inplace=True)

    return result

amostra_classificada_v_conformidade = check_circular_trips(amostra_classificada_v_conformidade)

# Uso:
# new_df = process_dataframe(amostra_classificada_v_conformidade)

In [13]:
amostra_classificada_v_conformidade.to_excel('./../data/treated/mudanca_circulares.xlsx')

In [14]:
# Convert the 'data' column to datetime format
amostra_classificada_v_conformidade['data'] = pd.to_datetime(amostra_classificada_v_conformidade['data'])

# Filter for rows where 'data' is in 2022
amostra_2022 = amostra_classificada_v_conformidade[amostra_classificada_v_conformidade['data'].dt.year == 2022]

# Filter for rows where 'data' is in 2023
amostra_2023 = amostra_classificada_v_conformidade[amostra_classificada_v_conformidade['data'].dt.year == 2023]


# Count non-NaN values in the 'status' column for each filtered DataFrame
non_nan_count_2022 = amostra_2022['status'].notna().sum()
non_nan_count_2023 = amostra_2023['status'].notna().sum()

print(f"Viagens classificadas em 2022: {non_nan_count_2022}")
print(f"Viagens classificadas em 2022 em %: {round(non_nan_count_2022 / amostra_classificada_v_conformidade.shape[0],2)*100} %")

print(f"Viagens classificadas em 2023: {non_nan_count_2023}")
print(f"Viagens classificadas em 2023: {round(non_nan_count_2023 / amostra_classificada_v_conformidade.shape[0],2)*100} %")

Viagens classificadas em 2022: 42
Viagens classificadas em 2022 em %: 40.0 %
Viagens classificadas em 2023: 24
Viagens classificadas em 2023: 23.0 %


Foram identificadas 33 das 104 viagens válidas da amostra, considerando que a LECD50 não teve viagens planejadas ou sinal de GPS após 2023-05-31 e para as datas da amostra de 2023 foram usados os dados da linha 605. Nestes casos, o veículo que realizou a viagem é o mesmo veículo que consta no amostra. 

A linha não teve viagem planejada em junho. Elas começaram em 16-07-2022.

Em julho, as viagens completas foram identificadas apenas nos dias 25, 26, 27 e 28, mesmo assim o POD foi zero no mês (explicação para o gráfico do POD).

<!-- 
SELECT
  *
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA BETWEEN "2022-06-01"
  AND "2022-07-31"
  AND servico = 'LECD50'

  Verificar se houveram mudanças no shape entre novembro e março. -->

#### 3.3 Verificar dados de GPS

In [15]:
amostra_com_status = amostra_classificada_v_conformidade[pd.notna(amostra_classificada_v_conformidade['status'])]
amostra_sem_status = amostra_classificada_v_conformidade[pd.isna(amostra_classificada_v_conformidade['status'])]
amostra_sem_status

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
32,2022-09-13,LECD50,27684,06:05:00,06:26:00,I,2022-09-13 06:05:00,2022-09-13 06:26:00,NaN,NaN,NaN,NaN,NaT,NaT
33,2022-09-13,LECD50,27684,06:27:00,06:51:00,V,2022-09-13 06:27:00,2022-09-13 06:51:00,NaN,NaN,NaN,NaN,NaT,NaT
34,2022-09-13,LECD50,27684,09:55:00,10:22:00,I,2022-09-13 09:55:00,2022-09-13 10:22:00,NaN,NaN,NaN,NaN,NaT,NaT
35,2022-09-13,LECD50,27684,10:24:00,10:49:00,V,2022-09-13 10:24:00,2022-09-13 10:49:00,NaN,NaN,NaN,NaN,NaT,NaT
46,2022-09-21,LECD50,27684,06:05:00,06:30:00,I,2022-09-21 06:05:00,2022-09-21 06:30:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2022-09-29,LECD50,27684,21:43:00,22:00:00,V,2022-09-29 21:43:00,2022-09-29 22:00:00,NaN,NaN,NaN,NaN,NaT,NaT
87,2022-09-29,LECD50,27684,22:06:00,22:21:00,I,2022-09-29 22:06:00,2022-09-29 22:21:00,NaN,NaN,NaN,NaN,NaT,NaT
88,2022-09-29,LECD50,27684,22:21:00,22:38:00,V,2022-09-29 22:21:00,2022-09-29 22:38:00,NaN,NaN,NaN,NaN,NaT,NaT
95,2023-07-12,LECD50,27632,08:04:00,08:42:00,I,2023-07-12 08:04:00,2023-07-12 08:42:00,NaN,NaN,NaN,NaN,NaT,NaT


In [16]:
data_n_encontradas = amostra_sem_status['data'].unique()
data_n_encontradas = pd.Series(data_n_encontradas)
data_n_encontradas = data_n_encontradas.dt.strftime('%Y-%m-%d')
data_n_encontradas = data_n_encontradas.values
data_n_encontradas

array(['2022-09-13', '2022-09-21', '2022-09-29', '2023-07-12'],
      dtype=object)

In [17]:
ids_n_encontrados = amostra_sem_status['id_veiculo_amostra'].unique()
ids_n_encontrados  

array(['27684', '27632'], dtype=object)

Verificar se existem dados de gps em `gps_sppo`.

In [18]:
data_n_encontradas_gps = amostra_sem_status['data'].unique()
data_n_encontradas_gps = pd.Series(data_n_encontradas_gps)
data_n_encontradas_gps = data_n_encontradas_gps.dt.strftime('%Y-%m-%d')
data_n_encontradas_gps = data_n_encontradas_gps.values
data_n_encontradas_gps

ids_n_encontrados_gps = amostra_sem_status['id_veiculo_amostra'].unique()

q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA IN {tuple(data_n_encontradas_gps)}
  AND id_veiculo IN ('A27684', 'A27632')
"""
          
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()
dados_gps['id_veiculo'] = dados_gps['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)

Downloading: 100%|██████████| 15429/15429 [00:01<00:00, 10130.62rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15429 entries, 0 to 15428
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           15429 non-null  object        
 1   servico              15429 non-null  object        
 2   timestamp_gps        15429 non-null  datetime64[ns]
 3   posicao_veiculo_geo  15429 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 482.3+ KB


In [19]:
def set_status(row, df_check):
    # Filter the df_check by vehicle ID and time range
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]
    
    # If there are rows in the filtered dataframe, check the condition
    if not filtered_df.empty:
        if (filtered_df.iloc[0]['servico'] == row['servico_amostra']):
            return "Encontrado sinal de GPS para o dia e horário indicados na amostra."
        else:
            unique_services = filtered_df['servico'].unique()
            return "Sinal de GPS encontrado para serviço diferente: {}".format(", ".join(map(str, unique_services)))
    else:
        return "Sem sinal de GPS para o veículo no dia e horário informados"

amostra_sem_status['status'] = amostra_sem_status.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_sem_status

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
32,2022-09-13,LECD50,27684,06:05:00,06:26:00,I,2022-09-13 06:05:00,2022-09-13 06:26:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
33,2022-09-13,LECD50,27684,06:27:00,06:51:00,V,2022-09-13 06:27:00,2022-09-13 06:51:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
34,2022-09-13,LECD50,27684,09:55:00,10:22:00,I,2022-09-13 09:55:00,2022-09-13 10:22:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
35,2022-09-13,LECD50,27684,10:24:00,10:49:00,V,2022-09-13 10:24:00,2022-09-13 10:49:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
46,2022-09-21,LECD50,27684,06:05:00,06:30:00,I,2022-09-21 06:05:00,2022-09-21 06:30:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2022-09-29,LECD50,27684,21:43:00,22:00:00,V,2022-09-29 21:43:00,2022-09-29 22:00:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
87,2022-09-29,LECD50,27684,22:06:00,22:21:00,I,2022-09-29 22:06:00,2022-09-29 22:21:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
88,2022-09-29,LECD50,27684,22:21:00,22:38:00,V,2022-09-29 22:21:00,2022-09-29 22:38:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
95,2023-07-12,LECD50,27632,08:04:00,08:42:00,I,2023-07-12 08:04:00,2023-07-12 08:42:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT


In [20]:
# juntar todas as linhas e classificar abaixo:
df_final = pd.concat([amostra_com_status, amostra_sem_status], ignore_index=True)
df_final

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2022-09-13,LECD50,27684,10:54:00,11:21:00,I,2022-09-13 10:54:00,2022-09-13 11:21:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 11:02:51,2022-09-13 11:42:51
1,2022-09-13,LECD50,27684,11:23:00,11:48:00,V,2022-09-13 11:23:00,2022-09-13 11:48:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 11:02:51,2022-09-13 11:42:51
2,2022-09-13,LECD50,27684,11:53:00,12:19:00,I,2022-09-13 11:53:00,2022-09-13 12:19:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 11:59:51,2022-09-13 12:38:51
3,2022-09-13,LECD50,27684,12:21:00,12:42:00,V,2022-09-13 12:21:00,2022-09-13 12:42:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 11:59:51,2022-09-13 12:38:51
4,2022-09-13,LECD50,27684,12:45:00,13:13:00,I,2022-09-13 12:45:00,2022-09-13 13:13:00,Viagem circular identificada e já paga,27684,LECD50,C,2022-09-13 12:52:21,2022-09-13 13:34:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2022-09-29,LECD50,27684,21:43:00,22:00:00,V,2022-09-29 21:43:00,2022-09-29 22:00:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
100,2022-09-29,LECD50,27684,22:06:00,22:21:00,I,2022-09-29 22:06:00,2022-09-29 22:21:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
101,2022-09-29,LECD50,27684,22:21:00,22:38:00,V,2022-09-29 22:21:00,2022-09-29 22:38:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
102,2023-07-12,LECD50,27632,08:04:00,08:42:00,I,2023-07-12 08:04:00,2023-07-12 08:42:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT


Padronizar o status após a verificação dos dados de GPS:

In [21]:
# Alterar o status para linhas 1
ids_to_change = [69, 73]
new_status_message = "Sinal de GPS encontrado para serviço diferente: 432"
df_final.loc[ids_to_change, 'status'] = new_status_message

# Alterar o status para linhas 
ids_to_change = [66, 67, 70, 71]
new_status_message = "Sinal de GPS fora do raio de 500m do início ou fim da viagem"
df_final.loc[ids_to_change, 'status'] = new_status_message

# Alterar o status para as linhas
ids_to_change = [102, 103]
new_status_message = "Viagem circular inválida - não atendeu ao percentual de conformidade do GPS ou do trajeto"
df_final.loc[ids_to_change, 'status'] = new_status_message

# Inserir os valores da viagem conformidade que não foi identificada:
filtered_df = viagem_conformidade[viagem_conformidade['datetime_partida'] == '2023-07-12T08:24:23']

# Obtendo os valores da única linha em filtered_df
values_from_filtered = filtered_df.iloc[0][['id_veiculo', 'servico', 'sentido', 'datetime_partida', 'datetime_chegada']].values

# Substituindo os valores em df_final para as linhas 102 e 103
df_final.loc[ids_to_change, ['id_veiculo_apurado', 'servico_apurado', 'sentido_apurado', 'datetime_partida_apurado', 'datetime_chegada_apurado']] = values_from_filtered

In [22]:
# exportar em csv
df_final.to_csv('./../data/output/analise_amostra_pre_solucao.csv', index=False)
df_final.to_excel('./../data/output/analise_amostra_pre_solucao.xlsx')

# Exemplos (mapas)

Viagem não identificada

Veículo: A27684 - LECD50

partida: 2022-09-13 06:05:00	
chegada: 2022-09-13 06:51:00	

In [23]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-13"
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.38rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [24]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2022-09-13"
  AND servico = 'LECD50'
  AND timestamp_gps BETWEEN '2022-09-13 06:05:00'
  AND '2022-09-13 06:51:00'
  AND id_veiculo = 'A27684'
"""
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 68/68 [00:00<00:00, 165.04rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           68 non-null     object        
 1   servico              68 non-null     object        
 2   timestamp_gps        68 non-null     datetime64[ns]
 3   posicao_veiculo_geo  68 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.2+ KB


In [25]:
map = create_trip_map(gps_identificado, shape_identificado)
map.save('./../data/figures/maps/A27684_LECD50_13_09_2022.html')
map

Viagem não identificada

A27684	21-09-2022 entre 06:05 e 06:49 - LECD50


In [26]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-21"
  AND servico = 'LECD50'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.82rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [27]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2022-09-21"
  AND servico = 'LECD50'
  AND timestamp_gps BETWEEN "2022-09-21T06:05:00"
  AND "2022-09-21T06:49:00"
  AND id_veiculo = "A27684"
"""

gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 69/69 [00:00<00:00, 163.12rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           69 non-null     object        
 1   servico              69 non-null     object        
 2   timestamp_gps        69 non-null     datetime64[ns]
 3   posicao_veiculo_geo  69 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.3+ KB


In [28]:
map = create_trip_map(gps_identificado, shape_identificado)
map.save('./../data/figures/maps/A27684_LECD50_21_09_2022.html')
map

Viagem não identificada

A27632	17-07-2023 entre 08:04 e 09:12 - 605

In [29]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-07-17"
  AND servico = '605'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.02rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [30]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2023-07-17"
  AND servico = '605'
  AND timestamp_gps BETWEEN "2023-07-17T08:04:00"
  AND "2023-07-17T09:12:00"
  AND id_veiculo = "A27632"
"""

gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 136/136 [00:00<00:00, 404.77rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           136 non-null    object        
 1   servico              136 non-null    object        
 2   timestamp_gps        136 non-null    datetime64[ns]
 3   posicao_veiculo_geo  136 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 4.4+ KB


In [31]:
map = create_trip_map(gps_identificado, shape_identificado)
map.save('./../data/figures/maps/A27632_605_17_07_2023.html')
map

## (2) Gráficos da análise exploratória / análise histórica da linha

- Gráfico POD até a data mais recente (quando fazemos a avaliação): `analise_pod_historico.png`

In [32]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '{servico}'
  AND DATA BETWEEN '2022-06-01' AND '2023-08-31'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 276/276 [00:00<00:00, 788.57rows/s]


In [33]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [34]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: {}".format(servico), font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

In [35]:
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])

# Criar o gráfico de linha com Plotly Express
fig = px.line(pod_pre_reprocessamento, 
              x='data', 
              y='perc_km_planejada', 
              title='Percentual de KM Planejada ao Longo do Tempo', 
              labels={'data': 'Data', 'perc_km_planejada': 'Percentual de KM Planejada'}, 
              markers=True)

# Mostrar o gráfico
fig.show()

Em dezembro as viagens são identificadas corretamente. Vale ressaltar que neste mês houveram viagens apenas até o dia 14 de dezembro (confirmado via dados de GPS).

In [113]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-12-14" 
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.93rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [117]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-12-14"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2022-12-14T14:00:00"
  AND "2022-12-14T14:42:00"
  AND id_veiculo = "A27684"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 56/56 [00:00<00:00, 140.35rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           56 non-null     object        
 1   servico              56 non-null     object        
 2   timestamp_gps        56 non-null     datetime64[ns]
 3   posicao_veiculo_geo  56 non-null     object        
 4   status_viagem        56 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.3+ KB


In [118]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

Com a mudança do shape no dia 01/01/2023, a viagem passou a não ser mais identificada corretamente. O problema persiste até hoje. 

Março de 2023:

Viagem encontrada no período afetado:
Mesmo as viagens identificadas, foram identificadas de forma errada.

In [73]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-03-22" 
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.54rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [82]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-03-22"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2023-03-22T09:49:00"
  AND "2023-03-22T10:40:00"
  AND id_veiculo = "A27556"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 281/281 [00:00<00:00, 745.36rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           281 non-null    object        
 1   servico              281 non-null    object        
 2   timestamp_gps        281 non-null    datetime64[ns]
 3   posicao_veiculo_geo  281 non-null    object        
 4   status_viagem        281 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 11.1+ KB


In [83]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

Viagem não encontrada no período afetado

In [84]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-03-22"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2023-03-22T06:14:00"
  AND "2023-03-22T06:51:00"
  AND id_veiculo = "A27556"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 135/135 [00:00<00:00, 266.27rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           135 non-null    object        
 1   servico              135 non-null    object        
 2   timestamp_gps        135 non-null    datetime64[ns]
 3   posicao_veiculo_geo  135 non-null    object        
 4   status_viagem        135 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 5.4+ KB


In [85]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

Fevereiro de 2023:

A27565

2023-02-06T06:19:26

2023-02-06T06:57:26

In [86]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-02-06"  
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.90rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [87]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-02-06"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2023-02-06T06:19:26"
  AND "2023-02-06T06:57:26"
  AND id_veiculo = "A27565"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 31/31 [00:00<00:00, 87.82rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           31 non-null     object        
 1   servico              31 non-null     object        
 2   timestamp_gps        31 non-null     datetime64[ns]
 3   posicao_veiculo_geo  31 non-null     object        
 4   status_viagem        31 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.3+ KB


In [88]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

03/04/2023 é quando o POD volta a normalidade, mas o problema persiste. É importante destacar que mesmo as viagens presentes na tabela de viagem completa não são válidas, pois não pegam o trajeto inteiro.

In [89]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-04-03"  
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.43rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [90]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-04-03"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2023-04-03T06:13:59"
  AND "2023-04-03T06:56:29"
  AND id_veiculo = "A27556"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 245/245 [00:00<00:00, 621.83rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           245 non-null    object        
 1   servico              245 non-null    object        
 2   timestamp_gps        245 non-null    datetime64[ns]
 3   posicao_veiculo_geo  245 non-null    object        
 4   status_viagem        245 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 9.7+ KB


In [91]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

31/05/2023 último dia com shape da LECD50

In [92]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-05-31"  
  AND servico = 'LECD50'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.87rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [93]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-05-31"
  AND servico_informado = 'LECD50'
  AND timestamp_gps BETWEEN "2023-05-31T06:12:24"
  AND "2023-05-31T09:51:54"
  AND id_veiculo = "A27556"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 36/36 [00:00<00:00, 87.59rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           36 non-null     object        
 1   servico              36 non-null     object        
 2   timestamp_gps        36 non-null     datetime64[ns]
 3   posicao_veiculo_geo  36 non-null     object        
 4   status_viagem        36 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.5+ KB


In [94]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

29/09/2023 com a 605 (data mais atual, o problema persiste).

In [ ]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-09-29"  
  AND servico = '605'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

In [95]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-09-29"
  AND servico_informado = '605'
  AND timestamp_gps BETWEEN "2023-09-29T06:13:32"
  AND "2023-09-29T06:49:02"
  AND id_veiculo = "A27632"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 44/44 [00:00<00:00, 118.60rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           44 non-null     object        
 1   servico              44 non-null     object        
 2   timestamp_gps        44 non-null     datetime64[ns]
 3   posicao_veiculo_geo  44 non-null     object        
 4   status_viagem        44 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.8+ KB


In [96]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map

Viagem identificada em 2023 na linha 605

In [37]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-07-04"  
  AND servico = '605'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.00rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [40]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-07-04"
  AND servico_informado = '605'
  AND timestamp_gps BETWEEN "2023-07-04T06:12:00"
  AND "2023-07-04T06:54:00"
  AND id_veiculo = "A27632"
  """
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 42/42 [00:00<00:00, 122.45rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           42 non-null     object        
 1   servico              42 non-null     object        
 2   timestamp_gps        42 non-null     datetime64[ns]
 3   posicao_veiculo_geo  42 non-null     object        
 4   status_viagem        42 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.8+ KB


In [41]:
map = create_trip_map(gps_identificado, shape_identificado)
# map.save('./../data/figures/maps/555.html')
map